In [2]:
# import torch as th

import matplotlib.pyplot as plt
import time

import sys
import os
from __future__ import division
import os
import urllib, cStringIO
import pymongo as pm ## first establish ssh tunnel to server where database is running
import base64
import numpy as np
from numpy import *
import PIL
from PIL import Image
import base64
import matplotlib
%matplotlib inline
from matplotlib import pylab, mlab, pyplot
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')
from matplotlib.path import Path
import matplotlib.patches as patches
import cv2
import pandas as pd
from svgpathtools import parse_path
import svg_distance_helpers as rsh
# import torch
# from torch.autograd import Variable
# import SimpleITK as sitk
# from torch.autograd import Variable


#### File Hierarchy

In [3]:
# directory & file hierarchy
analysis_dir = os.getcwd()
data_dir = os.path.join(analysis_dir,"tracing_overlap")
ref_dir = os.path.join(analysis_dir, 'tracing_ref_400')
tracing = ['square','shape', 'this circle']

#### Set Up

In [4]:
# generate reference shapes
really_run_this = 1
if really_run_this:
    ref_path = []
    for t in tracing:
        print t
        fname = os.path.join(ref_dir, '{}.png'.format(t))
        img_ref = cv2.imread(fname)
        img_ref = rsh.color_to_num(img_ref, threshold=0, white_background=False)
        ofname = os.path.join(ref_dir, '{}_ref.png'.format(t))
        cv2.imwrite(ofname, 255-img_ref*255)
        ref_path.append(ofname)
    

square
shape
this circle


In [ ]:
# Overlap the drawing on reference images
iteration_name = ['CDM_photodraw_e1', 'cdm_run_v3', 'cdm_run_v4']# name labels used in kiddraw database
dir_name = ['photodraw_e1', 'museumstation_v3', 'museumstation_v4'] # name labels will be used for new data storage 
tracing = ['square', 'shape', 'this circle']

# create the folder to store overlapped images
overlap_dir = os.path.join(analysis_dir, 'tracing_overlap')
if not os.path.exists(overlap_dir):
    os.makedirs(overlap_dir)

for di, dname in enumerate(dir_name):
    # create an individual folder for each experiment+version pair
    iter_dir = os.path.join(overlap_dir, dname)
    if not os.path.exists(iter_dir):
        os.makedirs(iter_dir)
        
    current_data_dir = os.path.join(analysis_dir,"tracing_{}".format(iteration_name[di]))

    # get tracings from the data folder
    for ti, t in enumerate(tracing):
            
        # folders that store the original tracing files
        if dname == 'photodraw_e1' and (t == 'square' or t == 'shape'):
            t = 'this ' + t
        t_dir = os.path.join(current_data_dir, t)
        
        # create folders to store overlapped images
        overlap_t = os.path.join(iter_dir, t.replace('this ', ''))
        if not os.path.exists(overlap_t):
                os.makedirs(overlap_t)
        
        
        if os.path.exists(t_dir):
            for fname in os.listdir(t_dir):
                if fname.startswith('age'):
                    img_draw = os.path.join(t_dir, fname)
                    img_ref = ref_path[ti]

                    draw = cv2.imread(img_draw).astype(int16)
                    ref = cv2.imread(img_ref).astype(int16)
                    img = ref
                    img[np.sum(ref, axis=2)<255*3] = [150, 150, 150]
                    img[np.sum(draw, axis=2)<255*3] = [40, 40, 170]

                    split_fname = fname.split('_')
                    age = split_fname[0]
                    timestamp = split_fname[3][2:]
                    new_fname = "{}_{}_{}_{}.png".format(age, dname, timestamp, t.replace('this ', ''))
                    cv2.imwrite(os.path.join(iter_dir,t.replace('this ', ''), new_fname), img)

#### upload stims to s3

Execute python script at command line by running: `python upload_stims_to_s3.py --bucket_name kiddraw-tracing --path_to_imgs ./tracing_eval_stims`

#### build stimulus dictionary & upload metadata to mongo

In [22]:
# read in data from csv files
cdm_v3 = pd.read_csv('MuseumStation_Tracing_Descriptives_cdm_run_v3.csv')
cdm_v4 = pd.read_csv('MuseumStation_Tracing_Descriptives_cdm_run_v4.csv')
photodraw = pd.read_csv('Photodraw_Tracing_Descriptives_CDM_photodraw_e1.csv')
limit = 200
overall = photodraw

for t in tracing:
    for a in range(2,11):
        cdm_v3_tage = cdm_v3.loc[(cdm_v3['age'] == 'age{}'.format(a)) & (cdm_v3['category']== t)]
        v3_size = cdm_v3_tage.shape[0]                 
        if v3_size > limit:
            overall = pd.concat([overall, cdm_v3_tage[:200]])
            
        else:
            overall = pd.concat([overall, cdm_v3_tage]) 
            
            cdm_v4_tage = cdm_v4.loc[(cdm_v4['age'] == 'age{}'.format(a)) & (cdm_v4['category']== t)]
            
            overall = pd.concat([overall, cdm_v4_tage[:limit-v3_size]])

print overall.shape
for t in tracing:
    for a in range(2,11):
        print t, a, overall.loc[(overall['age'] == 'age{}'.format(a)) & ( (overall['category'] == t) | (overall['category'] == 'this {}'.format(t)) )].shape

overall = overall.drop(['filename', "Unnamed: 0"], axis=1)
overall.reset_index(inplace = True)

(5251, 14)
square 2 (200, 14)
square 3 (200, 14)
square 4 (212, 14)
square 5 (224, 14)
square 6 (215, 14)
square 7 (209, 14)
square 8 (200, 14)
square 9 (169, 14)
square 10 (200, 14)
shape 2 (200, 14)
shape 3 (200, 14)
shape 4 (211, 14)
shape 5 (224, 14)
shape 6 (215, 14)
shape 7 (209, 14)
shape 8 (200, 14)
shape 9 (137, 14)
shape 10 (200, 14)
this circle 2 (200, 14)
this circle 3 (200, 14)
this circle 4 (213, 14)
this circle 5 (224, 14)
this circle 6 (215, 14)
this circle 7 (159, 14)
this circle 8 (155, 14)
this circle 9 (95, 14)
this circle 10 (165, 14)


In [23]:
overall['exp_name'] = 0
overall['iteration_name'] = 0
overall['timestamp'] = 0
overall['img_url'] = 0
overall['number_rating_levels'] = 5
overall['lower_bound'] = 'poor'
overall['upper_bound'] = 'excellent' 

for index, row in overall.iterrows():
    sid = row['session_id']    
    
    if sid.startswith('CDM'):
        overall.loc[index,'exp_name'] = 'photodraw'
        overall.loc[index,'iteration_name'] = 'e1'
    else:
        overall.loc[index,'exp_name'] = 'museumstation'
        overall.loc[index,'iteration_name'] = sid.split('_')[2][:2]
    
    timestamp = row['session_id'].split('_')[-1][2:]
    overall.loc[index, 'timestamp'] = timestamp
    category = row['category'].replace('this ','')
    overall.loc[index,'category'] = category
    
    fname = '{}_{}_{}_{}_{}.png'.format(row['age'], overall.loc[index,'exp_name'],overall.loc[index,'iteration_name'], timestamp, category)
    
    folder_name = '{}_{}'.format(overall.loc[index,'exp_name'], overall.loc[index,'iteration_name'])
    render_fname = os.path.join(data_dir, folder_name, category, fname)
    if not os.path.exists(render_fname):
        print index, fname
        overall = overall.drop(index)
        continue
    
    overall.loc[index,'img_url'] = 'https://s3.amazonaws.com/{}/{}'.format('kiddraw-tracing',fname)
    overall.loc[index,'games'] = '[]'
    overall.loc[index,'age'] = overall.loc[index,'age'].replace('age','')

overall['age'] = pd.to_numeric(overall['age'])


3 age4_photodraw_e1_1532113637303_square.png
27 age4_photodraw_e1_1532467421953_square.png
28 age4_photodraw_e1_1532467421953_circle.png
60 age4_photodraw_e1_1532547528521_shape.png
76 age4_photodraw_e1_1532555203616_shape.png
81 age5_photodraw_e1_1532556623648_square.png
83 age5_photodraw_e1_1532556623648_circle.png
108 age5_photodraw_e1_1533079562976_square.png
112 age6_photodraw_e1_1533079944053_shape.png
113 age6_photodraw_e1_1533079944053_circle.png
115 age4_photodraw_e1_1533080485289_shape.png
122 age4_photodraw_e1_1533145985106_circle.png
136 age5_photodraw_e1_1533148939408_shape.png
159 age4_photodraw_e1_1533158505451_square.png
161 age4_photodraw_e1_1533158505451_circle.png
175 age5_photodraw_e1_1533162144527_shape.png
178 age4_photodraw_e1_1533164692587_shape.png
179 age4_photodraw_e1_1533164692587_circle.png
182 age2_museumstation_v3_1525892637212_square.png
183 age2_museumstation_v3_1525892849063_square.png
188 age2_museumstation_v3_1525901322190_square.png
190 age2_museums

641 age4_museumstation_v3_1527970799228_square.png
643 age4_museumstation_v3_1527976799200_square.png
649 age4_museumstation_v3_1528154999081_square.png
653 age4_museumstation_v3_1528316999098_square.png
674 age4_museumstation_v3_1528581292853_square.png
679 age4_museumstation_v3_1528662866503_square.png
686 age4_museumstation_v3_1528845188306_square.png
689 age4_museumstation_v3_1528914599379_square.png
690 age4_museumstation_v3_1528915199199_square.png
691 age4_museumstation_v3_1528915252445_square.png
692 age4_museumstation_v3_1528915373007_square.png
697 age4_museumstation_v3_1529092030289_square.png
701 age4_museumstation_v3_1529177399237_square.png
702 age4_museumstation_v3_1529180568272_square.png
703 age4_museumstation_v3_1529180673021_square.png
707 age4_museumstation_v3_1529187760457_square.png
712 age4_museumstation_v3_1529264521476_square.png
713 age4_museumstation_v3_1529264999203_square.png
715 age4_museumstation_v3_1529271233301_square.png
717 age4_museumstation_v3_15292

1863 age10_museumstation_v3_1532120748535_square.png
1886 age10_museumstation_v3_1532804263628_square.png
1887 age10_museumstation_v3_1532822033299_square.png
1890 age10_museumstation_v3_1533151204661_square.png
1903 age10_museumstation_v3_1533321692454_square.png
1911 age10_museumstation_v3_1533340131947_square.png
1916 age10_museumstation_v3_1533680021857_square.png
1926 age10_museumstation_v3_1533936362193_square.png
1929 age10_museumstation_v3_1533936674158_square.png
1933 age10_museumstation_v3_1534028921827_square.png
1942 age10_museumstation_v3_1534285449019_square.png
1952 age2_museumstation_v3_1525897661521_shape.png
1953 age2_museumstation_v3_1525901322190_shape.png
1954 age2_museumstation_v3_1525990258467_shape.png
1955 age2_museumstation_v3_1526064952740_shape.png
1957 age2_museumstation_v3_1526075895791_shape.png
1960 age2_museumstation_v3_1526160796383_shape.png
1963 age2_museumstation_v3_1526241319670_shape.png
1967 age2_museumstation_v3_1526411112241_shape.png
1968 age2

2311 age3_museumstation_v3_1529697599412_shape.png
2313 age3_museumstation_v3_1529702920509_shape.png
2314 age3_museumstation_v3_1529702999538_shape.png
2318 age3_museumstation_v3_1529706210435_shape.png
2321 age3_museumstation_v3_1530381781760_shape.png
2322 age3_museumstation_v3_1530382112100_shape.png
2323 age3_museumstation_v3_1530384619314_shape.png
2324 age3_museumstation_v3_1530384772531_shape.png
2328 age3_museumstation_v3_1530398323737_shape.png
2329 age3_museumstation_v3_1530475738016_shape.png
2331 age3_museumstation_v3_1530480737675_shape.png
2333 age3_museumstation_v3_1530483520783_shape.png
2334 age3_museumstation_v3_1530483825626_shape.png
2335 age3_museumstation_v3_1530484721039_shape.png
2337 age3_museumstation_v3_1530562825622_shape.png
2338 age3_museumstation_v3_1530566132538_shape.png
2339 age3_museumstation_v3_1530566593366_shape.png
2341 age3_museumstation_v3_1530651257351_shape.png
2344 age3_museumstation_v3_1530725596187_shape.png
2348 age3_museumstation_v3_1530

2996 age7_museumstation_v3_1529186999108_shape.png
2999 age7_museumstation_v3_1529438632128_shape.png
3001 age7_museumstation_v3_1529440297346_shape.png
3003 age7_museumstation_v3_1529703938903_shape.png
3004 age7_museumstation_v3_1530310809450_shape.png
3011 age7_museumstation_v3_1530644143472_shape.png
3021 age7_museumstation_v3_1530818736526_shape.png
3036 age7_museumstation_v3_1531693076482_shape.png
3050 age7_museumstation_v3_1532457555949_shape.png
3067 age7_museumstation_v3_1533077875433_shape.png
3074 age7_museumstation_v3_1533414542620_shape.png
3080 age7_museumstation_v3_1533762023918_shape.png
3090 age7_museumstation_v3_1534195654681_shape.png
3103 age7_museumstation_v3_1534966969603_shape.png
3110 age7_museumstation_v3_1535322732324_shape.png
3111 age7_museumstation_v3_1535824080024_shape.png
3116 age7_museumstation_v3_1535920734002_shape.png
3152 age8_museumstation_v3_1526150370058_shape.png
3159 age8_museumstation_v3_1526666895291_shape.png
3160 age8_museumstation_v3_1527

3972 age3_museumstation_v3_1528489799206_circle.png
3977 age3_museumstation_v3_1528578938334_circle.png
3982 age3_museumstation_v3_1528660089959_circle.png
3986 age3_museumstation_v3_1528837199127_circle.png
3987 age3_museumstation_v3_1528841058316_circle.png
3988 age3_museumstation_v3_1528841172383_circle.png
3992 age3_museumstation_v3_1528917240117_circle.png
3994 age3_museumstation_v3_1528926256078_circle.png
3998 age3_museumstation_v3_1529015886007_circle.png
3999 age3_museumstation_v3_1529170799283_circle.png
4000 age3_museumstation_v3_1529171302239_circle.png
4003 age3_museumstation_v3_1529178374382_circle.png
4004 age3_museumstation_v3_1529178655413_circle.png
4007 age3_museumstation_v3_1529188199108_circle.png
4024 age3_museumstation_v3_1529533231799_circle.png
4025 age3_museumstation_v3_1529617447735_circle.png
4028 age3_museumstation_v3_1529695799951_circle.png
4034 age3_museumstation_v3_1529704801381_circle.png
4035 age3_museumstation_v3_1529706001344_circle.png
4039 age3_mu

In [24]:
## apply weird literal eval hack to convert string into regular list
import ast
overall['games'] = '[]'
overall['games'] = overall['games'].apply(lambda x: ast.literal_eval(x))
print overall.shape

(4266, 21)


In [25]:
try:
    overall = overall.drop('index', axis=1)
except:
    pass

## save out data frame as csv for our own records
overall.to_csv('kiddraw_tracing_eval.csv',index=False)

In [26]:
## convert 'overall' pandas dataframe to json object and save out
import json
stimdict = overall.to_dict(orient='records') 
with open('{}.js'.format('kiddraw_tracing_eval'), 'w') as fout:
    json.dump(stimdict, fout)

### load JSON back in to inspect before uploading to mongo

In [27]:
### define datasetname
dataset_name = 'kiddraw_tracing_eval'

### next todo is to upload this JSON to initialize the new stimulus collection
print('next todo is to upload this JSON to initialize the new stimulus collection...')
import json
J = json.loads(open('{}.js'.format(dataset_name),mode='ru').read())

next todo is to upload this JSON to initialize the new stimulus collection...


In [28]:
##assert len(J)==len(all_files)
print 'dataset_name: {}'.format(dataset_name)
print len(J)

dataset_name: kiddraw_tracing_eval
4266


In [29]:
## look at one example record
J[0]

{u'age': 7,
 u'bounding_box': u'(96, 287, 115, 300)',
 u'category': u'square',
 u'draw_duration_new': 0.0,
 u'draw_duration_old': 0.0,
 u'exp_name': u'photodraw',
 u'games': [],
 u'img_url': u'https://s3.amazonaws.com/kiddraw-tracing/age7_photodraw_e1_1532111112405_square.png',
 u'iteration_name': u'e1',
 u'lower_bound': u'poor',
 u'mean_intensity': 0.01055,
 u'num_strokes': 1,
 u'number_rating_levels': 5,
 u'session_id': u'CDM_photodraw_e11532111112405',
 u'submit_date': u'2018-07-20T19:04:03.404Z',
 u'submit_time': 1532113443404.0,
 u'timestamp': u'1532111112405',
 u'trial_duration': nan,
 u'trial_num': 0,
 u'upper_bound': u'excellent'}

In [41]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['stimuli']
coll = db[dataset_name]

In [42]:
coll.drop()
print coll.count()

0


In [45]:
from IPython.display import clear_output
## actually add data now to the database
reallyRun = 1
if reallyRun:
    for (i,j) in enumerate(J):
        if i%1==0:
            print ('%d of %d' % (i,len(J)))
            clear_output(wait=True)
        coll.insert(j)

4265 of 4266


In [48]:
## check how many records have been retrieved
a = coll.find({'num_strokes':{'$gte':0}}).sort('timestamp')
numGames = []
for rec in a:
    numGames.append(len(rec['games']))
b = np.array(numGames)
print np.mean(b>0)

0.0


In [47]:
coll.find_one()

{u'_id': ObjectId('5c0334fd1943e64087564cb9'),
 u'age': 7,
 u'bounding_box': u'(96, 287, 115, 300)',
 u'category': u'square',
 u'draw_duration_new': 0.0,
 u'draw_duration_old': 0.0,
 u'exp_name': u'photodraw',
 u'games': [],
 u'img_url': u'https://s3.amazonaws.com/kiddraw-tracing/age7_photodraw_e1_1532111112405_square.png',
 u'iteration_name': u'e1',
 u'lower_bound': u'poor',
 u'mean_intensity': 0.01055,
 u'num_strokes': 1,
 u'number_rating_levels': 5,
 u'session_id': u'CDM_photodraw_e11532111112405',
 u'submit_date': u'2018-07-20T19:04:03.404Z',
 u'submit_time': 1532113443404.0,
 u'timestamp': u'1532111112405',
 u'trial_duration': nan,
 u'trial_num': 0,
 u'upper_bound': u'excellent'}